In [ ]:
"""
This notebook is to visualise the maximum activation of the filter
Model wil use MLP with conditional time series data
"""


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# from keras.layers import Dense, Dropout, Activation, LSTM, Convolution1D, MaxPooling1D, Flatten
# from keras.models import Sequential
import matplotlib.pyplot as plt

%matplotlib inline
from torchbearer import Trial
import os

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision.datasets import MNIST
from torchbearer import Trial
import torchvision.transforms as transforms

In [ ]:
# define baseline MLP model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)        
        out = self.fc2(out)
        out = F.sigmoid(out)
        out = out.squeeze()
        return out

In [ ]:
# build the model and load state
model = BaselineModel()
model.load_state_dict(torch.load('mlp_model'))

# put model in eval mode
model.eval() 

In [ ]:
class gradient_generator():
    
    def __init__(self, model):
        
        self.loss = nn.BCELoss()
        self.model = model.to('cpu')
        self.gradients = None
        # Put model in evaluation mode
        self.model.eval()
        # Hook the first layer to get the gradient
        self.hook = self.hook_layers()
    def hook_layers(self):    
        def hook_function(module, grad_in, grad_out):
            print("hook called")
            print(grad_out[0])
            self.gradients = grad_out[0]
        first_layer = list(self.model._modules.items())[0][1]
        hook = first_layer.register_backward_hook(hook_function)
        return hook
    def generate_gradient(self, inputs, labels):
        self.model.zero_grad()
        _ = model(inputs)
        loss = self.loss(inputs, labels)
        loss.backward()
        
        self.hook.remove()
        return self.gradients
    

grad = torch.zeros((2412,400))
i = 0


for data in trainloader:
    # get the inputs
    inputs, labels = data
    print(inputs.shape, labels.shape)
    GG = gradient_generator(model)
    temp = GG.generate_gradient(inputs, labels)
    grad[i,:] = temp
    i += 1

grad